In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from src.utils.submission import createSubmissionFile
%matplotlib inline 


In [22]:
DATA_FOLDER = '../data/'
CUSTOM_DATA_FOLDER = '../data_custom/'
SUBMISSION_FOLDER = "../submissions/"

In [ ]:
transactions    = pd.read_csv(os.path.join(os.getcwd(), DATA_FOLDER, 'sales_train_v2.csv'))
items           = pd.read_csv(os.path.join(os.getcwd(), DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(os.getcwd(), DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(os.getcwd(), DATA_FOLDER, 'shops.csv'))
test            = pd.read_csv(os.path.join(os.getcwd(), DATA_FOLDER, 'test.csv'))

In [4]:
transactions['month'] = transactions.date.apply(lambda x: int(x.split(".")[1]))
transactions['day'] = transactions.date.apply(lambda x: int(x.split(".")[0]))
transactions['year'] = transactions.date.apply(lambda x: int(x.split(".")[2]))
transactions['day_revenue'] = transactions.item_price * transactions.item_cnt_day

In [27]:
transactions.info()
transactions.to_csv(os.path.join(os.getcwd(), CUSTOM_DATA_FOLDER, 'sales_train_v2.csv'), index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 10 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
 6   month           int64  
 7   day             int64  
 8   year            int64  
 9   day_revenue     float64
dtypes: float64(3), int64(6), object(1)
memory usage: 224.0+ MB


In [19]:
test.shop_id.unique()

array([ 5,  4,  6,  3,  2,  7, 10, 12, 28, 31, 26, 25, 22, 24, 21, 15, 16,
       18, 14, 19, 42, 50, 49, 53, 52, 47, 48, 57, 58, 59, 55, 56, 36, 37,
       35, 38, 34, 46, 41, 44, 39, 45], dtype=int64)

In [6]:
month_sold = transactions.groupby(["date_block_num", "shop_id", "item_id"]).item_cnt_day.sum().to_frame().reset_index()

In [26]:
month_sold = month_sold.rename(columns = {"item_cnt_day": "item_cnt_month"})
month_sold.to_csv(os.path.join(os.getcwd(), CUSTOM_DATA_FOLDER, 'month_sold_train.csv'), index=False)

In [28]:
last_month = 33
last_month_sold = month_sold[month_sold.date_block_num == last_month]
last_month_sold.to_csv(os.path.join(os.getcwd(), CUSTOM_DATA_FOLDER, 'last_month_sold_train.csv'), index=False)

In [9]:
merged_result = pd.merge(test, last_month_sold, "left", on=["shop_id", "item_id"])

In [10]:
merged_result = merged_result.fillna(0)

In [11]:
merged_result.head()

,ID,shop_id,item_id,date_block_num,item_cnt_month
0,0,5,5037,0.0,0.0
1,1,5,5320,0.0,0.0
2,2,5,5233,33.0,1.0
3,3,5,5232,0.0,0.0
4,4,5,5268,0.0,0.0


In [12]:
merged_result[merged_result.item_cnt_month > 20]

,ID,shop_id,item_id,date_block_num,item_cnt_month
460,460,5,20949,33.0,43.0
5374,5374,4,17717,33.0,25.0
5560,5560,4,20949,33.0,98.0
10333,10333,6,10201,33.0,78.0
10352,10352,6,10202,33.0,35.0
...,...,...,...,...,...
205331,205331,39,21811,33.0,46.0
205490,205490,39,17270,33.0,34.0
209233,209233,45,10201,33.0,26.0
209252,209252,45,10202,33.0,22.0


In [13]:
createSubmissionFile(merged_result, "last_month_or_0.csv")

In [16]:
last_year_sold = month_sold[month_sold.date_block_num == (last_month-11)]
merged_result_last_year = pd.merge(test, last_year_sold, "left", on=["shop_id", "item_id"])
merged_result_last_year = merged_result_last_year.fillna(0)
createSubmissionFile(merged_result_last_year, "last_year_same_month_or_0.csv")

In [29]:

merged_result_last_year

,ID,shop_id,item_id,date_block_num,item_cnt_month
0,0,5,5037,22.0,1.0
1,1,5,5320,0.0,0.0
2,2,5,5233,0.0,0.0
3,3,5,5232,0.0,0.0
4,4,5,5268,0.0,0.0
...,...,...,...,...,...
214195,214195,45,18454,0.0,0.0
214196,214196,45,16188,0.0,0.0
214197,214197,45,15757,0.0,0.0
214198,214198,45,19648,0.0,0.0
